In [34]:
import pandas as pd
import numpy as np
import os
import re
import pickle
from bs4 import BeautifulSoup as soup
import requests as rq
import time
import h5py
import scipy as sp
import scipy.stats

# EDA

## Pulling in Data

The data collected is a mix of HDF5, csv and web scraped data about the Bay area.  The CSVs are for particle concentraions, HDF5s for CO2 ppm, and the web scraped data is for empty lot/apartments for sale in the San Fransisco.

In [213]:
#starting off by reading in the csv files
aqcsvs = []
aqcsvs += ['../../../../Documents/largecsvs/others/'+each for each in os.listdir('../../../../Documents/largecsvs/pothers/') if each.endswith('.csv')]

part = []
part += ['../../../../Documents/largecsvs/particle/'+each for each in os.listdir('../../../../Documents/largecsvs/particle/') if each.endswith('csv')]
#getting all H5 files for the project
results = []
results += ['../../../../Downloads/'+each for each in os.listdir('../../../../Downloads/') if each.endswith('.h5')]

In [214]:
aqcsvs=sorted(aqcsvs)
part=sorted(part)

In [99]:
weatherdf = pd.read_csv('weather/1367384.csv')

In [105]:
weatherdf.drop(['STATION', 'NAME','ELEVATION', 'SNWD', 'TOBS'], axis=1, inplace=True)
weatherdf = weatherdf.fillna(0)

In [110]:
weatherdf.head()

LATITUDE  LONGITUDE        DATE  PRCP  SNOW  TAVG  TMAX  TMIN
0   37.7705  -122.4269  2014-01-01   0.0   0.0   0.0  63.0  44.0
1   37.7705  -122.4269  2014-01-02   0.0   0.0   0.0  63.0  48.0
2   37.7705  -122.4269  2014-01-03   0.0   0.0   0.0  64.0  49.0
3   37.7705  -122.4269  2014-01-04   0.0   0.0   0.0  66.0  46.0
4   37.7705  -122.4269  2014-01-05   0.0   0.0   0.0  66.0  48.0

In [221]:
aqcsvs

['COdaily_42101_2014.csv',
 'COdaily_42101_2015.csv',
 'COdaily_42101_2016.csv',
 'COdaily_42101_2017.csv',
 'COdaily_42101_2018.csv',
 'Odaily_44201_2014.csv',
 'Odaily_44201_2015.csv',
 'Odaily_44201_2016.csv',
 'Odaily_44201_2017.csv',
 'Odaily_44201_2018.csv',
 'SOdaily_42401_2014.csv',
 'SOdaily_42401_2015.csv',
 'SOdaily_42401_2016.csv',
 'SOdaily_42401_2017.csv',
 'SOdaily_42401_2018.csv',
 'daily_aqi_by_cbsa_2014.csv',
 'daily_aqi_by_cbsa_2015.csv',
 'daily_aqi_by_cbsa_2016.csv',
 'daily_aqi_by_cbsa_2017.csv',
 'daily_aqi_by_cbsa_2018.csv']

In [233]:
aq=pd.read_csv(aqcsvs[0])

for i in range(1,len(aqcsvs)):
    if i==15:
        aq2=pd.read_csv(aqcsvs[i])
    elif i>14:
        temp = pd.read_csv(aqcsvs[i])
        aq2 = pd.concat([aq2,temp], ignore_index=True)
    else:
        temp = pd.read_csv(aqcsvs[i])
        aq = pd.concat([aq,temp], ignore_index=True)

In [234]:
aq.head()

State Code  County Code  Site Num  Parameter Code  POC   Latitude  \
0           1           73        23           42101    2  33.553056   
1           1           73        23           42101    2  33.553056   
2           1           73        23           42101    2  33.553056   
3           1           73        23           42101    2  33.553056   
4           1           73        23           42101    2  33.553056   

   Longitude  Datum   Parameter Name Sample Duration         ...          AQI  \
0    -86.815  WGS84  Carbon monoxide          1 HOUR         ...          NaN   
1    -86.815  WGS84  Carbon monoxide          1 HOUR         ...          NaN   
2    -86.815  WGS84  Carbon monoxide          1 HOUR         ...          NaN   
3    -86.815  WGS84  Carbon monoxide          1 HOUR         ...          NaN   
4    -86.815  WGS84  Carbon monoxide          1 HOUR         ...          NaN   

  Method Code                                        Method Name  \
0        93.0  INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER   
1        93.0  INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER   
2        93.0  INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER   
3        93.0  INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER   
4        93.0  INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER   

    Local Site Name                                Address  State Name  \
0  North Birmingham  NO. B'HAM,SOU R.R., 3009 28TH ST. NO.     Alabama   
1  North Birmingham  NO. B'HAM,SOU R.R., 3009 28TH ST. NO.     Alabama   
2  North Birmingham  NO. B'HAM,SOU R.R., 3009 28TH ST. NO.     Alabama   
3  North Birmingham  NO. B'HAM,SOU R.R., 3009 28TH ST. NO.     Alabama   
4  North Birmingham  NO. B'HAM,SOU R.R., 3009 28TH ST. NO.     Alabama   

   County Name   City Name              CBSA Name  Date of Last Change  
0    Jefferson  Birmingham  Birmingham-Hoover, AL           2016-08-26  
1    Jefferson  Birmingham  Birmingham-Hoover, AL           2016-08-26  
2    Jefferson  Birmingham  Birmingham-Hoover, AL           2016-08-26  
3    Jefferson  Birmingham  Birmingham-Hoover, AL           2016-08-26  
4    Jefferson  Birmingham  Birmingham-Hoover, AL           2016-08-26  

[5 rows x 29 columns]

In [223]:
#Filtering out unneed data
aq = aq[aq['State Name']=='California']
aq = aq[aq['CBSA Name']=='San Francisco-Oakland-Hayward, CA']
aq = aq[aq['City Name'].isin(['Oakland','Richmond','San Pablo','San Francisco','San Rafael'])]
aq.drop(['State Code', 'County Code', 'Site Num', 'Parameter Code', 'POC', 'Datum', 'Sample Duration',
       'Pollutant Standard', 'Units of Measure', 'Event Type',
       'Observation Count', 'Observation Percent', 'AQI', 'Method Code', 'Method Name',
       'Local Site Name', 'Address', 'State Name', 'County Name', 'City Name', 'Date of Last Change'], axis=1, inplace=True)
#filtering to relevant cities
aq.reset_index(drop=True,inplace=True)

In [224]:
codf = aq[aq['Parameter Name']=='Carbon monoxide']
ozdf= aq[aq['Parameter Name']=='Ozone']
sodf= aq[aq['Parameter Name']=='Ozone']


In [232]:
aq2.head()

CBSA  CBSA Code        Date  AQI  \
134147  San Francisco-Oakland-Hayward, CA      41860  2014-01-01  109   
134148  San Francisco-Oakland-Hayward, CA      41860  2014-01-02   80   
134149  San Francisco-Oakland-Hayward, CA      41860  2014-01-03   80   
134150  San Francisco-Oakland-Hayward, CA      41860  2014-01-04   67   
134151  San Francisco-Oakland-Hayward, CA      41860  2014-01-05   56   

                              Category Defining Parameter Defining Site  \
134147  Unhealthy for Sensitive Groups              PM2.5   06-001-0011   
134148                        Moderate              PM2.5   06-001-0011   
134149                        Moderate              PM2.5   06-001-0011   
134150                        Moderate              PM2.5   06-001-0011   
134151                        Moderate              PM2.5   06-001-0007   

        Number of Sites Reporting  
134147                         12  
134148                         13  
134149                         12  
134150                         12  
134151                         13

In [229]:
#Filtering out unneed data
aq2 = aq2[aq2['CBSA']=='San Francisco-Oakland-Hayward, CA']
aq2 = aq2[aq2['CSBA'].isin(['Oakland, CA','Richmond, CA','San Pablo,CA','San Francisco, CA','San Rafael, CA'])]
aq2.drop(['CBSA', 'CBSA Code', 'Date', 'AQI', 'Category', 'Defining Parameter',
       'Defining Site', 'Number of Sites Reporting'], axis=1, inplace=True)
#filtering to relevant cities
aq2.reset_index(drop=True,inplace=True)
aq2.head()

KeyError: 'CSBA'

In [ ]:
codf.columns=['Latitude', 'Longitude', 'Param Name', 'Date Local',
       'CO Mean', 'CO Max Value', 'CO Max Hour']
ozdf.columns=['Latitude', 'Longitude', 'Param Name2', 'Date Local',
       'Oz Mean', 'Oz Max Value', 'Oz Max Hour']
sodf.columns=['Latitude', 'Longitude', 'Param Name3', 'Date Local',
       'SO Mean', 'SO Max Value', 'SO Max Hour']
aq2.columns=['Latitude', 'Longitude', 'Response', 'Date Local',
       'AQ Mean', 'AQ Max Value', 'AQ Max Hour']


In [ ]:
aq = pd.merge(codf,ozdf, on=['Latitude', 'Longitude', 'Date Local'])
aq = pd.merge(aq,sodf, on=['Latitude', 'Longitude', 'Date Local'])
aq = pd.merge(aq,aq2, on=['Latitude', 'Longitude', 'Date Local'])

In [ ]:
aq.head()

In [22]:
def h5ToDF(file):
    f = h5py.File(file, 'r')
    date = [pd.to_datetime(str(x).replace('b\'','').replace('\'','')) for x in list(f['RetrievalHeader']['retrieval_time_string'])]
    latlong = [(x,y) for x,y in zip(f['RetrievalGeometry']['retrieval_latitude_geoid'],f['RetrievalGeometry']['retrieval_longitude_geoid'])]
    co2ppm = list(f['RetrievalResults']['xco2'])
    co2_uncert = [x*10**6 for x in list(f['RetrievalResults']['xco2_uncert'])]
    df = pd.DataFrame({'date':date,
        'latlong': latlong,
        'co2ppm': co2ppm,
        'co2_uncert': co2_uncert
    })
    return df
                                                                                    
                                                                                    
                                                                                    

In [23]:
df = f = h5py.File(results[0], 'r')
date = [pd.to_datetime(str(x).replace('b\'','').replace('\'','')) for x in list(f['RetrievalHeader']['retrieval_time_string'])]
latlong = [(x,y) for x,y in zip(f['RetrievalGeometry']['retrieval_latitude_geoid'],f['RetrievalGeometry']['retrieval_longitude_geoid'])]
co2ppm = list(f['RetrievalResults']['xco2'])
co2_uncert = [x*10**6 for x in list(f['RetrievalResults']['xco2_uncert'])]
df = pd.DataFrame({'date':date,
                'latlong': latlong,
                'co2ppm': co2ppm,
                'co2_uncert': co2_uncert})
for i in range(1,len(results)):
    df.append(h5ToDF(results[i]))

In [24]:
df['co2ppm'] = [x*10**6 for x in df['co2ppm']]

In [28]:
df['co2_uncert'] = [float(x) for x in df['co2_uncert']]

In [29]:
df.head()

co2_uncert      co2ppm                    date                 latlong
0    0.886318  402.186939 2016-11-13 20:26:38.363   (-83.56647, 38.90449)
1    0.786782  398.468226 2016-11-13 20:26:38.390  (-83.59027, 39.013676)
2    0.780767  396.430114 2016-11-13 20:26:38.417   (-83.61452, 39.12222)
3    0.777430  398.074626 2016-11-13 20:26:38.562  (-83.46712, 38.238407)
4    0.799400  399.525161 2016-11-13 20:26:38.616   (-83.51127, 38.46135)

In [30]:
#Writing a pickle file to save data
import pickle
with open('my_data.pkl', 'wb') as picklefile:
    pickle.dump(df, picklefile)

In [ ]:
#list(f['RetrievalGeometry']['retrieval_latitude_geoid'])

## Web Scraping

Pulling data from loopnet.com for areas where we can greenify citys (make them more environmentally friendly). 

In [97]:
#initializing sites that I will pull from
flexspace = 'http://www.loopnet.com/california/san-francisco_flex-space-for-sale/'
land = 'http://www.loopnet.com/california/san-francisco_land-for-sale/'
apts = 'http://www.loopnet.com/california/san-francisco_apartment-buildings-for-sale/'

In [98]:
response = rq.get(land)
page = response.text
reviewp = soup(page,"lxml")


In [99]:
reviewp.find_all(class_='listingTitle')

[<span class="listingTitle">0.05 Acre Site for Proposed Apartments</span>,
 <span class="listingTitle">1228 Folsom Street</span>,
 <span class="listingTitle">1876 Oakdale Ave</span>,
 <span class="listingTitle">1791 Mission Street</span>,
 <span class="listingTitle">240 7th St</span>,
 <span class="listingTitle">1924 Mission St</span>,
 <span class="listingTitle">2435-2445 16th St</span>,
 <span class="listingTitle">Topaz Way</span>,
 <span class="listingTitle">Portfolio of 4 Properties</span>,
 <span class="listingTitle">1234 Shoreline Hwy</span>,
 <span class="listingTitle">81 Ervine St</span>,
 <span class="listingTitle">OCTAVIA M+N</span>,
 <span class="listingTitle">250 Church St</span>,
 <span class="listingTitle">Mission and 22nd</span>]

## Exploring

Exploring the data, kicking out points that are wrong based on the error and other stats.

In [33]:
print('Max: '+ str(max(df['co2_uncert'])))
print('Min: ' + str(min(df['co2_uncert'])))
print('Average: '+ str(np.average(df['co2_uncert'])))
print('Median: '+ str(np.median(df['co2_uncert'])))

Max: 13.534934623748995
Min: 0.1912240037427182
Average: 0.4512936642967768
Median: 0.4234372852351953


In [46]:
np.percentile(df['co2_uncert'], 99.5)

1.041737527884833

In [51]:
#Removing Outliers because of large error associated with the calculation
df= (df[df['co2_uncert']<1.04])